In [11]:
from ipynb.fs.full.feature_engineering_tree import X_Transformer_scaled
from ipynb.fs.full.feature_engineering_tree import X_Transformer
from ipynb.fs.full.feature_engineering_tree import DataLoader
from ipynb.fs.full.feature_engineering_tree import TargetNormalizedRegressor
from lightgbm import LGBMRegressor
from ipynb.fs.full.feature_engineering_final import BlendingRegressor
from sklearn.linear_model import LassoCV
from sklearn.linear_model import RidgeCV
from sklearn.pipeline import Pipeline
from xgboost import XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import ElasticNetCV
import xgboost as xgb
import numpy as np
import pandas as pd
from xgboost import plot_tree
from mlxtend.regressor import StackingCVRegressor
import matplotlib.pyplot as plt
%matplotlib inline
from xgboost import plot_tree
from matplotlib.pylab import rcParams
from sklearn.model_selection import KFold
from sklearn.svm import SVR

In [12]:
data_loader = DataLoader("train_imp_r.csv","test_imp_r.csv")

/home/victor/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:3508: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/home/victor/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:3538: PearsonRNearConstantInputWarning: An input array is nearly constant; the computed correlation coefficent may be inaccurate.
  warnings.warn(PearsonRNearConstantInputWarning())


In [13]:
X_train = data_loader.getX_train()
Y_train = data_loader.getY_train()
X_test = data_loader.getX_test()

In [14]:
x_transformer = X_Transformer()

/home/victor/anaconda3/lib/python3.6/site-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


In [15]:
kf = KFold(n_splits=10,shuffle = True,random_state = 27)

In [16]:
xgb = XGBRegressor(objective='reg:squarederror',eval_metric = 'rmse',learning_rate=0.02,n_estimators=3000,
                                     max_depth=4,
                                     gamma=0, subsample=0.7,
                                     colsample_bynode=0.14,
                                     nthread=-1,
                                     scale_pos_weight=1, seed=27,min_child_weight=15,
                                     reg_alpha=0.00006)

xgb_t = TargetNormalizedRegressor(xgb)

gbr = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.02,
                                      max_depth=4, max_features='sqrt', subsample=0.75,
                                      min_samples_leaf=15,
                                      loss='huber', random_state = 5)
gbr_t = TargetNormalizedRegressor(gbr)


In [17]:
learning_pipeline = Pipeline(steps=[('preprocess', x_transformer),('learn',gbr_t)])
learning_pipeline.fit(X_train,y=Y_train)





/home/victor/anaconda3/lib/python3.6/site-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)
/home/victor/anaconda3/lib/python3.6/site-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)
/home/victor/anaconda3/lib/python3.6/site-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


Pipeline(memory=None,
         steps=[('preprocess',
                 Pipeline(memory=None,
                          steps=[('col_drop',
                                  ColumnDrop(columns=['Utilities', 'Street',
                                                      'PoolQC'])),
                                 ('col_transformer',
                                  DfColumnTransformer(n_jobs=None,
                                                      remainder='drop',
                                                      sparse_threshold=0.3,
                                                      transformer_weights=None,
                                                      transformers=[('categorical_with_missing_values_const',
                                                                     PipelineWithFeatureNames(feature_names=['BsmtQ...
                                                                               loss='huber',
                                                

In [18]:
#set up the parameters
#rcParams['figure.figsize'] = 100,90
#plot_tree(learning_pipeline.named_steps['learn'],num_trees=2799, rankdir='LR')#plt.show()


In [19]:
Ypredicted = learning_pipeline.predict(X_test)
#Ypredicted =   #pd.read_csv('submission_stack_elasticnet.csv').iloc[:,1]
#Ypredicted = pd.read_csv('submission_stack_ridge.csv').iloc[:,1]*0.5 + pd.read_csv('submission_stack_elasticnet.csv').iloc[:,1]*0.5

In [20]:
resultDf = pd.DataFrame()
resultDf['Id'] = data_loader.get_Test_id()
resultDf['SalePrice'] = Ypredicted

#q1 = resultDf['SalePrice'].quantile(0.0045)
#q2 = resultDf['SalePrice'].quantile(0.99)
#resultDf[resultDf['SalePrice'] <= q1]
#resultDf[resultDf['SalePrice'] <= q1]

#resultDf['SalePrice'] = resultDf['SalePrice'].apply(lambda x: x if x > q1 else x*0.9)
#resultDf['SalePrice'] = resultDf['SalePrice'].apply(lambda x: x if x < q2 else x*1.081)

#resultDf
resultDf.to_csv('submission_gbr_new.csv',index = False)